###  Libraries

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_circles, make_moons
from sklearn.metrics.pairwise import pairwise_kernels
import matplotlib.pyplot as plt

from tslearn.clustering import KernelKMeans

from KernelKMeansPP import KernelKMeansPP

### Global Variables

In [ ]:
seed = 42

### Load Dataset

In [ ]:
n_samples = 500

X, y = make_circles(n_samples=n_samples, factor=0.2, noise=0.05, random_state=seed)
len(np.where(np.array(y) == 1)[0])

# Find indices of samples with label 1
indices_label_1 = np.where(np.array(y) == 0)[0]

# Define the number of samples to remove
num_samples_to_remove = 0

# Randomly select indices to remove
indices_to_remove = np.random.choice(indices_label_1, size=num_samples_to_remove, replace=False)

# Remove selected samples
X = np.delete(X, indices_to_remove, axis=0)
y = np.delete(y, indices_to_remove)

# Number of samples after removal
print("Number of samples after removal:", len(X))
#X, y = make_moons(n_samples=n_samples, noise=0.1, random_state=seed)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)

In [ ]:
K = 2 
kernel_matrix = pairwise_kernels(X, X, metric='rbf', gamma=5)

In [ ]:
kmeans = KMeans(n_clusters=2).fit(X)
plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels_)

In [ ]:
# Should find a way to init the fucking centers!
kernel_kmeans = KernelKMeans(n_clusters=2, n_init=1, kernel='precomputed').fit(kernel_matrix)
plt.scatter(X[:, 0], X[:, 1], c=kernel_kmeans.labels_)
plt.show()

In [ ]:
kernelKMeansPP =  KernelKMeansPP()
kernelKMeansPP.kernel_kmeans_pp(X, 2, kernel_matrix, method='KkMeans')